# Derivatives Exam - March 2023

Miguel Teodoro, 93127

In [133]:
import numpy as np
from scipy.stats import norm

In [134]:
N = norm.cdf

In [135]:
# strikes
s0a = 95
s0b = 94

# vols
sigma_a = 0.305
sigma_b = 0.303

# cont. compounded rf
r = 0.0405

# time to maturity
T = 187
Tp = 187/250

# spec. time relevant for some q's
t1 = 68
t1p = 68/250

# single day
dt = 1/250

# Q1.

Compute the price of an at-the-money European call option on stock A with maturity at time 𝑇.

In [132]:
# Calculate price of call option using black scholes formula

def bs_call(S_, X_, r_, Tp_, vol_):
    # Calculate d1
    d1 = (np.log(S_/X_)+(r_+(vol_**2)/2)*Tp_)/(vol_*np.sqrt(Tp_))
    # Calculate d2
    d2 = d1 - vol_ * np.sqrt(Tp_)
    # Calculate call value
    call = S_ * N(d1) - X_ * np.exp(-r_ * Tp_) * N(d2)
    
    print("S: ", S_)
    print("K: ", X_)
    print("r: ", r_)
    print("Tp: ", Tp_)
    print("sigma: ", vol_)
    print("d1: ", d1)
    print("d2: ", d2)
    print("N(d1): ",N(d1))
    print("N(d2): ",N(d2))
    
    return call

In [120]:
# Calculate call value

q1 = bs_call(s0a, s0a, r, Tp, sigma_a)
q1

S:  95
K:  95
r:  0.0405
Tp:  0.748
sigma:  0.305
d1:  0.24673604916224662
d2:  -0.017049280225117147
N(d1):  0.5974437471462545
N(d2):  0.49319865076976716


11.301391298201324

# Q2.

Compute the price of a European call option on stock A with maturity at time 𝑇, and 𝐾 being the lowest of: (i) the stock price at 𝑡0, and (ii) the stock price at 𝑡1.

In [99]:
# Define function to construct price paths according to Ito's Lemma

def price_paths(S0_, r_, sigma_, T_, dt_, n_paths):
    paths = []
    # For each path
    for iteration in np.arange(0,n_paths,1):
        current_price = S0_
        iteration_path = [current_price]
        z = norm.rvs(size = T_)
        # Apply a new list of prices
        for step in np.arange(1, T_, 1):
            current_price = current_price * np.exp((r_ - ((sigma_**2)/2)) * dt_ + z[step-1] * sigma_ * np.sqrt(dt_))
            iteration_path.append(current_price)
        # Join all paths in a list
        paths.append([iteration, iteration_path])
    return paths

In [100]:
# Function to search for average price at time T, might be useful later on

def average_price(paths_, time):
    sum = 0
    for path_ in range(0, len(paths_)):
        sum += paths_[path_][1][time-1]
    return sum/len(paths_)

In [101]:
# Construct price paths (adjustable number of iterations)

iter = 2000
paths_a = price_paths(s0a, r, sigma_a, T, dt, iter)

In [103]:
# Define function to calculate call payoff

def call_payoff(sT_, K_):
    return max(sT_ - K_, 0)

In [105]:
# To get the price of a call, we will simulate the payoff in each simulated path

def simulate_payoffs_q2(paths_, t_, T_, s0_):
    payoff_list = []
    for path in paths_:
        s0 = s0_
        st1 = path[1][t_-1]
        if s0 <= st1:
            k = s0
        else:
            k = st1
        sT = path[1][T_-1]
        pi = call_payoff(sT, k)
        payoff_list.append(pi)
    return payoff_list

In [106]:
discount_payoffs = np.exp(-r*Tp)

q2 = np.average(simulate_payoffs_q2(paths_a, t1, T, s0a))
print("Average Call value: ", q2*discount_payoffs)

Average Call value:  12.773866353944497


In [121]:
# Premium for option to switch k

premium_q2 = q2 - q1
premium_q2

1.8653676604856901

# Q3.

Compute the price of a European call option on stock A with maturity at time 𝑇, and 𝐾 being the lowest of: (i) the stock price at 𝑡0, and (ii) the stock price at 𝑡1; subject to a floor of 90% of the stock price at 𝑡0 (said differently, 𝐾 can’t be lower than 90% of the stock price at 𝑡0)

In [107]:
# To get the price of a call, we will simulate the payoff in each simulated path

def simulate_payoffs_q3(paths_, t_, T_, s0_):
    payoff_list = []
    for path in paths_:
        s0 = s0_
        st1 = path[1][t_-1]
        # If St1 below the threshold, then "cap" it
        if st1 < 0.9*s0:
            st1 = 0.9*s0
        # Compare and define k as the lowest
        if s0 <= st1:
            k = s0
        else:
            k = st1
        sT = path[1][T_-1]
        # Calculate payoff
        pi = call_payoff(sT, k)
        payoff_list.append(pi)
    return payoff_list

In [109]:
q3 = np.average(simulate_payoffs_q3(paths_a, t1, T, s0a))
print("Average Call value: ", q3*discount_payoffs)

Average Call value:  12.167641315493702


In [122]:
# Cost for "capping" k

cost_q3 = q2 - q3
cost_q3

0.6248710245459552

# Q4.

Consider the European call option on stock A described in question 3. Further consider an otherwise similar European call option written on stock B. Then, compute the price of a derivative that would allow you to choose, at time 𝑇, between: (i) the payoff from the option written on stock A, and (ii) the payoff from the option written on stock B.


In [110]:
def simulate_payoffs_q4(paths_a_, paths_b_, t_, T_, s0a_, s0b_):
    payoff_list = []
    for npath in range(0, len(paths_a)):
        
        # Simulate payoff for a
        path_a = paths_a_[npath]
        st1_a = path_a[1][t_-1]
        if st1_a < 0.9*s0a_:
            st1_a = 0.9*s0a_
        if s0a_ <= st1_a:
            ka = s0a_
        else:
            ka = st1_a
        sTa = path_a[1][T_-1]
        pi_a = call_payoff(sTa, ka)

        # Simulate payoff for b
        path_b = paths_b_[npath]
        st1_b = path_b[1][t_-1]
        if st1_b < 0.9*s0b_:
            st1_b = 0.9*s0b_
        if s0b_ <= st1_b:
            kb = s0b_
        else:
            kb = st1_b
        sTb = path_b[1][T_-1]
        pi_b = call_payoff(sTb, kb)

        # Keep the highest with max
        payoff_list.append(max(pi_a,pi_b))

    return payoff_list

In [111]:
# Construct price paths (adjustable number of iterations)

iter = 2000
paths_b = price_paths(s0b, r, sigma_b, T, dt, iter)

In [112]:
q4 = np.average(simulate_payoffs_q4(paths_a, paths_b, t1, T, s0a, s0b))
print("Average Call value: ", q4*discount_payoffs)

Average Call value:  20.763706326984686


# Q5.

Suppose you want to take a long position in the derivative described in question 4. Further suppose that an investment bank was willing to buy/sell this derivative at a price that was 10% higher than the value you computed in question 4 (for simplicity, assume no bid/ask spread). Does this represent an arbitrage opportunity? If so, how would you exploit it? If not, how would you justify this discrepancy?

## Average Arbitrage Strategy Payoff

In [113]:
# Pay for assets

pay_assets_t0 = -94-95
pay_assets_t0

-189

In [114]:
# Borrow loan

borrow = -pay_assets_t0
borrow

189

In [115]:
# Sell option

sell_option = q4*1.1
sell_option

23.54258136050918

In [116]:
# Pay back loan

pb_loan = borrow * (1+(r*Tp))
pb_loan

194.72556600000001

In [117]:
# Profit from holding assets
asseta_avg_payoff = average_price(paths_a, T) # - s0a
assetb_avg_payoff = average_price(paths_b, T) # - s0b
asset_avg_payoff = asseta_avg_payoff + assetb_avg_payoff
asset_avg_payoff

194.32204971196933

In [118]:
# Pay back the option
q4

21.402346691371978

In [119]:
t0profit = pay_assets_t0+borrow+sell_option
t1profit = -pb_loan+asset_avg_payoff-q4

total_profit = t0profit+t1profit

print("T0 \n")
print("Pay Assets: ", pay_assets_t0)
print("Borrow: ", borrow)
print("Sell option: ", sell_option)
print("-> T0 Profit = ", t0profit, "\n")
print("T1 \n")
print("Pay Back Loan: ", -pb_loan)
print("Asset Average Price: ", asset_avg_payoff)
print("Option average payoff: ", q4)
print("-> T1 Profit = ", t1profit, "\n")
print("-----")
print("Total Profit: ", total_profit)

T0 

Pay Assets:  -189
Borrow:  189
Sell option:  23.54258136050918
-> T0 Profit =  23.54258136050918 

T1 

Pay Back Loan:  -194.72556600000001
Asset Average Price:  194.32204971196933
Option average payoff:  21.402346691371978
-> T1 Profit =  -21.805862979402665 

-----
Total Profit:  1.7367183811065132


### Check if there is actual arbitrage, by using the strategy and simulating for all paths

In [129]:
def find_no_arbitrage(paths_a_, paths_b_, t_, T_, s0a_, s0b_):
    # For each simulated path
    found = 0
    while found == 0:

        for npath in range(0, len(paths_a)):
            
            # Calculate payoff a

            path_a = paths_a_[npath]
            st1_a = path_a[1][t_-1]
            if st1_a < 0.9*s0a_:
                st1_a = 0.9*s0a_
            if s0a_ <= st1_a:
                ka = s0a_
            else:
                ka = st1_a
            sTa = path_a[1][T_-1]
            pi_a = call_payoff(sTa, ka)

            # Calculate payoff b

            path_b = paths_b_[npath]
            st1_b = path_b[1][t_-1]
            if st1_b < 0.9*s0b_:
                st1_b = 0.9*s0b_
            if s0b_ <= st1_b:
                kb = s0b_
            else:
                kb = st1_b
            sTb = path_b[1][T_-1]
            pi_b = call_payoff(sTb, kb)

            # Calculate arbitrage strategy payoff

            option_payoff = max(pi_a,pi_b)
            pay_assets_t0 = -s0a-s0b
            borrow = -pay_assets_t0
            sell_option = q4*1.1
            pb_loan = borrow * (1+(r*Tp))
            asset_avg_payoff = path_a[1][T_-1] + path_b[1][T_-1]

            t0profit = pay_assets_t0+borrow+sell_option
            t1profit = -pb_loan+asset_avg_payoff-option_payoff

            total_profit = t0profit+t1profit

            # If finds a negative payoff

            if total_profit < 0:
                found = 1
                print("There is no arbitrage. Profit is negative under following conditions: ")
                print("T0 \n")
                print("Pay Assets: ", pay_assets_t0)
                print("Borrow: ", borrow)
                print("Sell option: ", sell_option)
                print("-> T0 Profit = ", t0profit, "\n")
                print("T1 \n")
                print("Pay Back Loan: ", -pb_loan)
                print("Asset Average Price: ", asset_avg_payoff)
                print("Option average payoff: ", option_payoff)
                print("-> T1 Profit = ", t1profit, "\n")
                print("-----")
                print("Total Profit: ", total_profit)
                print("ST a: ", sTa)
                print("ST b: ", sTb)
                
                break # Remove break if want to see all paths where payoff is negative.

    if found == 0:
        print("There is arbitrage.")


In [130]:
find_no_arbitrage(paths_a, paths_b, t1, T, s0a, s0b)

There is no arbitrage. Profit is negative under following conditions: 
T0 

Pay Assets:  -189
Borrow:  189
Sell option:  23.54258136050918
-> T0 Profit =  23.54258136050918 

T1 

Pay Back Loan:  -194.72556600000001
Asset Average Price:  164.76116449323865
Option average payoff:  11.105686966466678
-> T1 Profit =  -41.07008847322804 

-----
Total Profit:  -17.52750711271886
ST a:  69.05547752677198
ST b:  95.70568696646669
